# 10-4-1 ImageFolder (1)

In [1]:
import torchvision
from torchvision import transforms

from torch.utils.data import DataLoader

In [2]:
from matplotlib.pyplot import imshow
%matplotlib inline

In [3]:
# Compose : transformation 들을 하나로 묶어주는 역할을 한다.
trans = transforms.Compose([ 
    transforms.Resize((64,128)) # 사이즈를 새로 맞춰준다.
])
# train_data 에 내가 저장한 위치의 데이터를 저장하고
# transform 을 위에서 정의한 trans 의 방식으로 transform 하겠다! 
train_data = torchvision.datasets.ImageFolder(root='custom_data/origin_data', transform=trans)
# 그러나 여기에서 넘어가면 resize 를 또 해주어야한다.... 
# 그러면 여기서 변환을 바로 데이터셋에다 적용하면 되지 않을까??

In [4]:
for num, value in enumerate(train_data):
    data, label = value
    print(num, data, label)
    
    if(label == 0):
        # origin data 가 아니라 train data 로 ! 
        # 새로 resize 해서 저장을 하자.
        # 이거 하기전에 지정한 위치에 폴더를 만들어 주어야한다.!
        data.save('custom_data/train_data/gray/%d_%d.jpeg'%(num, label))
    else:
        data.save('custom_data/train_data/red/%d_%d.jpeg'%(num, label))

0 <PIL.Image.Image image mode=RGB size=128x64 at 0x1B0D6EB2408> 0
1 <PIL.Image.Image image mode=RGB size=128x64 at 0x1B0D75A40C8> 0
2 <PIL.Image.Image image mode=RGB size=128x64 at 0x1B0D75A4548> 0
3 <PIL.Image.Image image mode=RGB size=128x64 at 0x1B0D75A45C8> 0
4 <PIL.Image.Image image mode=RGB size=128x64 at 0x1B0D75A4648> 0
5 <PIL.Image.Image image mode=RGB size=128x64 at 0x1B0D75A46C8> 0
6 <PIL.Image.Image image mode=RGB size=128x64 at 0x1B0D75A4748> 0
7 <PIL.Image.Image image mode=RGB size=128x64 at 0x1B0D75A47C8> 0
8 <PIL.Image.Image image mode=RGB size=128x64 at 0x1B0D75A4848> 0
9 <PIL.Image.Image image mode=RGB size=128x64 at 0x1B0D75A48C8> 0
10 <PIL.Image.Image image mode=RGB size=128x64 at 0x1B0D75A4948> 0
11 <PIL.Image.Image image mode=RGB size=128x64 at 0x1B0D75A49C8> 0
12 <PIL.Image.Image image mode=RGB size=128x64 at 0x1B0D75A4A48> 0
13 <PIL.Image.Image image mode=RGB size=128x64 at 0x1B0D75A4AC8> 0
14 <PIL.Image.Image image mode=RGB size=128x64 at 0x1B0D75A4B48> 0
15 <P

# 실습

이제 같이 실습해 보아요~

In [5]:
import torch
import torch.nn as nn
import torch.nn.functional as F

import torch.optim as optim
from torch.utils.data import DataLoader

import torchvision
import torchvision.transforms as transforms

In [6]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(777)
if device =='cuda':
    torch.cuda.manual_seed_all(777)

In [7]:
# to tensor 로 바꾸어야 학습이 진행되므로 ! tensor 로 감싸주
trans = transforms.Compose([
    transforms.ToTensor()
])

train_data = torchvision.datasets.ImageFolder(root='./custom_data/train_data', transform=trans)

## 데이터로더

In [8]:
data_loader = DataLoader(dataset = train_data, batch_size = 8, shuffle = True, num_workers=2)

## 모델 형성

In [9]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(3,6,5),
            nn.ReLU(),
            nn.MaxPool2d(2),
        )
        self.layer2 = nn.Sequential(
            nn.Conv2d(6,16,5),
            nn.ReLU(),
            nn.MaxPool2d(2),
        )
        self.layer3 = nn.Sequential(
            nn.Linear(16*13*29, 120),
            nn.ReLU(),
            nn.Linear(120,2)
        )
        
    def forward(self, x):
        out = self.layer1(x)
        #print(out.shape) # 각 print 들은 우리가 test 할떄 보는것이다!
        out = self.layer2(out)
        #print(out.shape)
        out = out.view(out.shape[0], -1)
        #print(out.shape) # 확인하고 나면 꼭 지워주기~
        out = self.layer3(out)
        return out
        

## 모델 테스팅

모델을 몰떄에 네트워크에서 출력들이 어떤 형태로 변환되는지에 대해 볼 수 있따.

In [10]:
#testing 
# 위에서 print 를 넣은 상태에서 실행을 시키게 되면 우리가 넣은 torch.Tensor 에서
net = CNN().to(device)
test_input = (torch.Tensor(3,3,64,128)).to(device)
test_out = net(test_input)

## 세부사항 조절

In [11]:
optimizer = optim.Adam(net.parameters(), lr=0.00005)
loss_func = nn.CrossEntropyLoss().to(device)

In [22]:
total_batch = len(data_loader)

epochs = 3
for epoch in range(epochs):
    avg_cost = 0.0
    for num, data in enumerate(data_loader):
        imgs, labels = data
        imgs = imgs.to(device)
        labels = labels.to(device)
        optimizer.zero_grad()
        out = net(imgs)
        loss = loss_func(out, labels)
        loss.backward()
        optimizer.step()
        
        avg_cost += loss / total_batch
        
    print('[Epoch:{}] cost = {}'.format(epoch+1, avg_cost))
print('Learning Finished!')   

[Epoch:1] cost = 0.03486904501914978
[Epoch:2] cost = 0.01667075976729393
[Epoch:3] cost = 0.009321668185293674
Learning Finished!


## 모델저장 및 불러오기

In [14]:
# 모델을 저장합니다~ 
torch.save(net.state_dict(), "./model/model.pth")

In [15]:
# 똑같이 모델을 선언
new_net = CNN().to(device)

In [16]:
# 아까 저장한 값들을 불러오자.
new_net.load_state_dict(torch.load('./model/model.pth'))

<All keys matched successfully>

In [17]:
# 원래 net 과 new net 간의 값을 비교해보자~
print(net.layer1[0]) 
print(new_net.layer1[0])

print(net.layer1[0].weight[0][0][0])
print(new_net.layer1[0].weight[0][0][0])

net.layer1[0].weight[0] == new_net.layer1[0].weight[0]

Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
tensor([-0.0928,  0.0018, -0.0185, -0.0225,  0.0918], grad_fn=<SelectBackward>)
tensor([-0.0928,  0.0018, -0.0185, -0.0225,  0.0918], grad_fn=<SelectBackward>)


tensor([[[True, True, True, True, True],
         [True, True, True, True, True],
         [True, True, True, True, True],
         [True, True, True, True, True],
         [True, True, True, True, True]],

        [[True, True, True, True, True],
         [True, True, True, True, True],
         [True, True, True, True, True],
         [True, True, True, True, True],
         [True, True, True, True, True]],

        [[True, True, True, True, True],
         [True, True, True, True, True],
         [True, True, True, True, True],
         [True, True, True, True, True],
         [True, True, True, True, True]]])

## 모델 테스트

In [18]:
trans=torchvision.transforms.Compose([
    transforms.Resize((64,128)),
    transforms.ToTensor()
])
test_data = torchvision.datasets.ImageFolder(root='./custom_data/test_data', transform=trans)

In [19]:
test_set = DataLoader(dataset = test_data, batch_size = len(test_data))

In [21]:
with torch.no_grad(): # 테스트니까 no.grad~
    for num, data in enumerate(test_set):
        imgs, label = data
        imgs = imgs.to(device)
        label = label.to(device)
        
        prediction = net(imgs)
        
        correct_prediction = torch.argmax(prediction, 1) == label
        
        accuracy = correct_prediction.float().mean()
        print('Accuracy:', accuracy.item())

Accuracy: 1.0
